In [ ]:
### This file updated today to trace the gradio not visible

In [ ]:
# unified.py
# Unified trainer & predictor: 10 models (CNN/RNN x word/char/combined/fasttext_keras/fasttext_gensim)
# Save to /content/unified.py and run:
#   !python /content/unified.py --train cnn_word --csv "/content/drive/MyDrive/output_8_1_M_1_balanced_utf8.csv"
#   !python /content/unified.py --serve

import os
import re
import sys
import json
import argparse
import datetime
import traceback
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Embedding, Conv1D, GlobalMaxPooling1D,
                                     Dense, Dropout, concatenate, LSTM, Bidirectional)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from tensorflow.keras.utils import to_categorical

# gensim FastText (real)
from gensim.models import FastText
from gensim.utils import simple_preprocess

# Gradio
import gradio as gr

from sklearn.preprocessing import LabelEncoder

# -----------------------------
# Configuration (user-specified)
# -----------------------------
ROOT_MODEL_DIR = "/10models"
CSV_FALLBACK = "/output_8_1_M_1_balanced_utf8.csv"

# Paths for FastText storage (user-specified)
FT_GENSIM_DIR = "/10models/fasttest_gensim"
FT_KERAS_DIR = "/10models/fasttest_keras"

MODEL_NAMES = [
    "cnn_word",
    "cnn_char",
    "cnn_combined",
    "cnn_fasttext_keras",
    "cnn_fasttext_gensim",
    "rnn_word",
    "rnn_char",
    "rnn_combined",
    "rnn_fasttext_keras",
    "rnn_fasttext_gensim"
]

CFG = {
    "word": {"max_words": 20000, "max_len": 25, "embedding_dim": 100},
    "char": {"max_chars": 200, "vocab_size": 200, "embedding_dim": 64},
    "cnn": {"filters": [2, 3, 4], "num_filters": 128, "dropout": 0.5},
    "rnn": {"rnn_units": 128, "dropout": 0.5},
    "training": {"batch_size": 64, "epochs": 15, "validation_split": 0.15},
    # gensim FastText (real)
    "fasttext_gensim": {"vector_size": 300, "window": 5, "min_count": 1, "workers": 4, "epochs": 10, "sg": 1},
    # Keras-style fasttext embedding (trainable embedding layer, optional subword-like handling could be added)
    "fasttext_keras": {"embedding_dim": 100}
}

# -----------------------------
# Ensure directories
# -----------------------------
def ensure_dirs():
    os.makedirs(ROOT_MODEL_DIR, exist_ok=True)
    os.makedirs(FT_GENSIM_DIR, exist_ok=True)
    os.makedirs(FT_KERAS_DIR, exist_ok=True)
    for mn in MODEL_NAMES:
        os.makedirs(os.path.join(ROOT_MODEL_DIR, mn), exist_ok=True)

ensure_dirs()

# -----------------------------
# Paths & helpers
# -----------------------------
def model_folder(model_name):
    p = os.path.join(ROOT_MODEL_DIR, model_name)
    os.makedirs(p, exist_ok=True)
    return p

def model_paths(model_name):
    base = model_folder(model_name)
    return {
        "base": base,
        "model_best": os.path.join(base, "model_best.h5"),
        "model_epoch_pattern": os.path.join(base, "model_epoch-{epoch:02d}-val_loss-{val_loss:.4f}.h5"),
        "word_tokenizer": os.path.join(base, "word_tokenizer.json"),
        "char_tokenizer": os.path.join(base, "char_tokenizer.json"),
        "classes": os.path.join(base, "classes.npy"),
        "fasttext_gensim": os.path.join(FT_GENSIM_DIR, f"{model_name}_fasttext.model"),
        "fasttext_keras": os.path.join(FT_KERAS_DIR, f"{model_name}_ft_keras.json"),
        "training_state": os.path.join(base, "training_state.json"),
        "train_log": os.path.join(base, "train.log")
    }

def save_json_atomic(obj, path):
    tmp = path + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)
    os.replace(tmp, path)

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def append_log(model_name, msg):
    p = model_paths(model_name)["train_log"]
    ts = datetime.datetime.utcnow().isoformat() + "Z"
    with open(p, "a", encoding="utf-8") as f:
        f.write(f"[{ts}] {msg}\n")

# -----------------------------
# Training state (resume)
# -----------------------------
def load_training_state(model_name):
    p = model_paths(model_name)["training_state"]
    if os.path.exists(p):
        try:
            return load_json(p)
        except Exception:
            return {}
    return {}

def save_training_state(model_name, state):
    p = model_paths(model_name)["training_state"]
    save_json_atomic(state, p)

class EpochCheckpointCallback(Callback):
    def __init__(self, model_name):
        super().__init__()
        self.model_name = model_name
    def on_epoch_end(self, epoch, logs=None):
        state = load_training_state(self.model_name) or {}
        if "phases" not in state:
            state["phases"] = {}
        state["phases"][self.model_name] = {
            "last_completed_epoch": int(epoch) + 1,
            "updated_at": datetime.datetime.utcnow().isoformat() + "Z",
            "logs": (logs or {})
        }
        save_training_state(self.model_name, state)

# -----------------------------
# Data loading & preprocessing
# -----------------------------
def simple_clean_text(text):
    text = str(text)
    return re.sub(r"\s+", " ", text).strip()

def load_data(csv_path=""):
    if csv_path and os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
    elif os.path.exists(CSV_FALLBACK):
        df = pd.read_csv(CSV_FALLBACK)
    else:
        raise FileNotFoundError(f"CSV not found at {csv_path} or fallback {CSV_FALLBACK}")

    df = df.rename(columns={c: c.strip() for c in df.columns})
    if "trade_name" not in df.columns or "reason" not in df.columns:
        raise ValueError("CSV must contain 'trade_name' and 'reason' columns")
    df = df[["trade_name", "reason"]].dropna()
    df["trade_name"] = df["trade_name"].astype(str).apply(simple_clean_text)
    df["reason"] = df["reason"].astype(str).str.strip()
    return df

# -----------------------------
# Tokenizers & char mapping
# -----------------------------
def build_word_tokenizer(texts, max_words):
    tok = Tokenizer(num_words=max_words, oov_token="<OOV>")
    tok.fit_on_texts(texts)
    return tok

def save_tokenizer_json(tokenizer, path):
    with open(path, "w", encoding="utf-8") as f:
        f.write(tokenizer.to_json())

def load_tokenizer_json(path):
    from tensorflow.keras.preprocessing.text import tokenizer_from_json
    with open(path, "r", encoding="utf-8") as f:
        return tokenizer_from_json(f.read())

def build_char_tokenizer(texts, max_vocab=None):
    chars = set()
    for s in texts:
        for ch in s:
            chars.add(ch)
    chars = sorted(chars)
    if max_vocab is not None:
        chars = chars[:max_vocab-2]
    char_to_index = {ch: idx+2 for idx, ch in enumerate(chars)}
    char_to_index["<PAD>"] = 0
    char_to_index["<OOV>"] = 1
    return char_to_index

def save_char_tokenizer(char_map, vocab_size, path):
    save_json_atomic({"char_to_index": char_map, "vocab_size": vocab_size}, path)

def load_char_tokenizer(path):
    data = load_json(path)
    return data["char_to_index"], data.get("vocab_size", max(data["char_to_index"].values()) + 1)

def texts_to_char_sequences(texts, char_to_index, max_len):
    seqs = []
    pad = char_to_index.get("<PAD>", 0)
    oov = char_to_index.get("<OOV>", 1)
    for s in texts:
        arr = [char_to_index.get(ch, oov) for ch in s]
        if len(arr) < max_len:
            arr = arr + [pad] * (max_len - len(arr))
        else:
            arr = arr[:max_len]
        seqs.append(arr)
    return np.array(seqs, dtype=np.int32)

# -----------------------------
# FastText (gensim) helpers
# -----------------------------
def tokenize_for_fasttext(text):
    return simple_preprocess(text, deacc=False)

def train_fasttext_gensim(sentences, path, ft_conf):
    if not sentences or len(sentences) == 0:
        raise ValueError("FastText requires non-empty sentences")
    model = FastText(vector_size=ft_conf["vector_size"],
                     window=ft_conf["window"],
                     min_count=ft_conf["min_count"],
                     workers=ft_conf["workers"],
                     sg=ft_conf.get("sg", 1))
    model.build_vocab(sentences=sentences)
    model.train(sentences=sentences, total_examples=len(sentences), epochs=ft_conf["epochs"])
    model.save(path)
    return model

def load_fasttext_gensim(path):
    return FastText.load(path)

def text_to_word_vectors(ft_model, tokens, max_len):
    vsz = ft_model.vector_size
    vecs = []
    for t in tokens[:max_len]:
        try:
            v = ft_model.wv.get_vector(t)
        except Exception:
            v = np.zeros(vsz, dtype=np.float32)
        vecs.append(v)
    if len(vecs) < max_len:
        vecs.extend([np.zeros(vsz, dtype=np.float32)] * (max_len - len(vecs)))
    return np.array(vecs, dtype=np.float32)

# -----------------------------
# Model builders
# -----------------------------
def build_word_cnn(max_words, max_len, embedding_dim, n_classes, filters, num_filters, dropout):
    inp = Input(shape=(max_len,), name="word_input")
    emb = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len, name="word_emb")(inp)
    convs = []
    for f in filters:
        c = Conv1D(filters=num_filters, kernel_size=f, activation="relu")(emb)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    x = concatenate(convs) if len(convs) > 1 else convs[0]
    x = Dropout(dropout)(x)
    out = Dense(n_classes, activation="softmax")(x)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_char_cnn(vocab_size, max_chars, embedding_dim, filters, num_filters, dropout, n_classes):
    inp = Input(shape=(max_chars,), name="char_input")
    emb = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_chars, name="char_emb")(inp)
    convs = []
    for f in filters:
        c = Conv1D(filters=num_filters, kernel_size=f, activation="relu")(emb)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    x = concatenate(convs) if len(convs) > 1 else convs[0]
    x = Dropout(dropout)(x)
    out = Dense(n_classes, activation="softmax")(x)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_combined_cnn(word_conf, char_conf, n_classes):
    w_in = Input(shape=(word_conf["max_len"],), name="word_input")
    w_emb = Embedding(input_dim=word_conf["max_words"], output_dim=word_conf["embedding_dim"], input_length=word_conf["max_len"])(w_in)
    w_convs = []
    for f in CFG["cnn"]["filters"]:
        c = Conv1D(filters=CFG["cnn"]["num_filters"], kernel_size=f, activation="relu")(w_emb)
        c = GlobalMaxPooling1D()(c)
        w_convs.append(c)
    w_feat = concatenate(w_convs) if len(w_convs) > 1 else w_convs[0]
    w_feat = Dropout(CFG["cnn"]["dropout"])(w_feat)

    c_in = Input(shape=(char_conf["max_chars"],), name="char_input")
    c_emb = Embedding(input_dim=char_conf["vocab_size"], output_dim=char_conf["embedding_dim"], input_length=char_conf["max_chars"])(c_in)
    c_convs = []
    for f in char_conf.get("filters", [3,4,5]):
        c = Conv1D(filters=CFG["cnn"]["num_filters"], kernel_size=f, activation="relu")(c_emb)
        c = GlobalMaxPooling1D()(c)
        c_convs.append(c)
    c_feat = concatenate(c_convs) if len(c_convs) > 1 else c_convs[0]
    c_feat = Dropout(char_conf.get("dropout", CFG["char"].get("dropout", 0.5)))(c_feat)

    merged = concatenate([w_feat, c_feat])
    merged = Dropout(0.5)(merged)
    out = Dense(n_classes, activation="softmax")(merged)
    model = Model(inputs=[w_in, c_in], outputs=out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_fasttext_combined_cnn(ft_embed_dim, word_max_len, char_conf, n_classes):
    w_in = Input(shape=(word_max_len, ft_embed_dim), name="word_vec_input")
    convs = []
    for f in CFG["cnn"]["filters"]:
        c = Conv1D(filters=CFG["cnn"]["num_filters"], kernel_size=f, activation="relu")(w_in)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    w_feat = concatenate(convs) if len(convs) > 1 else convs[0]
    w_feat = Dropout(CFG["cnn"]["dropout"])(w_feat)

    c_in = Input(shape=(char_conf["max_chars"],), name="char_input")
    c_emb = Embedding(input_dim=char_conf["vocab_size"], output_dim=char_conf["embedding_dim"], input_length=char_conf["max_chars"])(c_in)
    c_convs = []
    for f in char_conf.get("filters", [3,4,5]):
        c = Conv1D(filters=CFG["cnn"]["num_filters"], kernel_size=f, activation="relu")(c_emb)
        c = GlobalMaxPooling1D()(c)
        c_convs.append(c)
    c_feat = concatenate(c_convs) if len(c_convs) > 1 else c_convs[0]
    c_feat = Dropout(char_conf.get("dropout", CFG["char"].get("dropout", 0.5)))(c_feat)

    merged = concatenate([w_feat, c_feat])
    merged = Dropout(0.5)(merged)
    out = Dense(n_classes, activation="softmax")(merged)
    model = Model(inputs=[w_in, c_in], outputs=out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# RNN builders
def build_word_rnn(max_words, max_len, embedding_dim, n_classes, rnn_units, dropout):
    inp = Input(shape=(max_len,), name="word_input")
    emb = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len)(inp)
    x = Bidirectional(LSTM(rnn_units))(emb)
    x = Dropout(dropout)(x)
    out = Dense(n_classes, activation="softmax")(x)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_char_rnn(vocab_size, max_chars, embedding_dim, rnn_units, dropout, n_classes):
    inp = Input(shape=(max_chars,), name="char_input")
    emb = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_chars)(inp)
    x = Bidirectional(LSTM(rnn_units))(emb)
    x = Dropout(dropout)(x)
    out = Dense(n_classes, activation="softmax")(x)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_combined_rnn(word_conf, char_conf, n_classes):
    w_in = Input(shape=(word_conf["max_len"],), name="word_input")
    w_emb = Embedding(input_dim=word_conf["max_words"], output_dim=word_conf["embedding_dim"], input_length=word_conf["max_len"])(w_in)
    w_x = Bidirectional(LSTM(CFG["rnn"]["rnn_units"]))(w_emb)
    w_x = Dropout(CFG["rnn"]["dropout"])(w_x)

    c_in = Input(shape=(char_conf["max_chars"],), name="char_input")
    c_emb = Embedding(input_dim=char_conf["vocab_size"], output_dim=char_conf["embedding_dim"], input_length=char_conf["max_chars"])(c_in)
    c_x = Bidirectional(LSTM(char_conf.get("rnn_units", CFG["rnn"]["rnn_units"])))(c_emb)
    c_x = Dropout(char_conf.get("dropout", CFG["rnn"]["dropout"]))(c_x)

    merged = concatenate([w_x, c_x])
    merged = Dropout(0.5)(merged)
    out = Dense(n_classes, activation="softmax")(merged)
    model = Model(inputs=[w_in, c_in], outputs=out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_fasttext_combined_rnn(ft_embed_dim, word_max_len, char_conf, n_classes):
    w_in = Input(shape=(word_max_len, ft_embed_dim), name="word_vec_input")
    w_x = Bidirectional(LSTM(CFG["rnn"]["rnn_units"]))(w_in)
    w_x = Dropout(CFG["rnn"]["dropout"])(w_x)

    c_in = Input(shape=(char_conf["max_chars"],), name="char_input")
    c_emb = Embedding(input_dim=char_conf["vocab_size"], output_dim=char_conf["embedding_dim"], input_length=char_conf["max_chars"])(c_in)
    c_x = Bidirectional(LSTM(char_conf.get("rnn_units", CFG["rnn"]["rnn_units"])))(c_emb)
    c_x = Dropout(char_conf.get("dropout", CFG["rnn"]["dropout"]))(c_x)

    merged = concatenate([w_x, c_x])
    merged = Dropout(0.5)(merged)
    out = Dense(n_classes, activation="softmax")(merged)
    model = Model(inputs=[w_in, c_in], outputs=out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# -----------------------------
# Central training function
# -----------------------------
def train_model(model_name, csv_path="", force_retrain=False):
    model_name = str(model_name).strip()
    if model_name not in MODEL_NAMES:
        raise ValueError("Unknown model: " + model_name)
    append_log(model_name, f"=== START TRAIN [{model_name}] ===")
    print(f"Starting training for: {model_name}")

    paths = model_paths(model_name)
    df = load_data(csv_path)
    texts = df["trade_name"].tolist()

    # labels
    le = LabelEncoder()
    y = le.fit_transform(df["reason"])
    classes = le.classes_
    n_classes = len(classes)
    np.save(paths["classes"], classes, allow_pickle=True)
    y_cat = to_categorical(y, num_classes=n_classes)

    state = load_training_state(model_name) or {}
    phases = state.get("phases", {})
    last_completed = int(phases.get(model_name, {}).get("last_completed_epoch", 0))
    initial_epoch = last_completed
    epochs = CFG["training"]["epochs"]

    cb_epoch = ModelCheckpoint(paths["model_epoch_pattern"], save_best_only=False, monitor="val_loss", mode="min", verbose=1)
    cb_best = ModelCheckpoint(paths["model_best"], save_best_only=True, monitor="val_loss", mode="min", verbose=1)
    cb_early = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, verbose=1)
    cb_state = EpochCheckpointCallback(model_name)

    try:
        # 1) cnn_word
        if model_name == "cnn_word":
            tok_path = paths["word_tokenizer"]
            if os.path.exists(tok_path) and not force_retrain:
                word_tok = load_tokenizer_json(tok_path)
            else:
                word_tok = build_word_tokenizer(texts, CFG["word"]["max_words"])
                save_tokenizer_json(word_tok, tok_path)
            seqs = word_tok.texts_to_sequences(texts)
            X_word = pad_sequences(seqs, maxlen=CFG["word"]["max_len"], padding="post", truncating="post")
            if os.path.exists(paths["model_best"]) and not force_retrain:
                model = load_model(paths["model_best"])
            else:
                model = build_word_cnn(CFG["word"]["max_words"], CFG["word"]["max_len"], CFG["word"]["embedding_dim"], n_classes, CFG["cnn"]["filters"], CFG["cnn"]["num_filters"], CFG["cnn"]["dropout"])
            if initial_epoch < epochs:
                model.fit(X_word, y_cat, batch_size=CFG["training"]["batch_size"], epochs=epochs, initial_epoch=initial_epoch, validation_split=CFG["training"]["validation_split"], callbacks=[cb_epoch, cb_best, cb_early, cb_state], verbose=2)

        # 2) cnn_char
        elif model_name == "cnn_char":
            char_path = paths["char_tokenizer"]
            if os.path.exists(char_path) and not force_retrain:
                char_map, vocab_size = load_char_tokenizer(char_path)
            else:
                char_map = build_char_tokenizer(texts, CFG["char"]["vocab_size"])
                vocab_size = max(char_map.values()) + 1
                save_char_tokenizer(char_map, vocab_size, char_path)
            X_char = texts_to_char_sequences(texts, char_map, CFG["char"]["max_chars"])
            if os.path.exists(paths["model_best"]) and not force_retrain:
                model = load_model(paths["model_best"])
            else:
                filters = CFG["char"].get("filters", [3,4,5])
                model = build_char_cnn(vocab_size, CFG["char"]["max_chars"], CFG["char"]["embedding_dim"], filters, CFG["cnn"]["num_filters"], CFG["char"].get("dropout", CFG["cnn"]["dropout"]), n_classes)
            if initial_epoch < epochs:
                model.fit(X_char, y_cat, batch_size=CFG["training"]["batch_size"], epochs=epochs, initial_epoch=initial_epoch, validation_split=CFG["training"]["validation_split"], callbacks=[cb_epoch, cb_best, cb_early, cb_state], verbose=2)

        # 3) cnn_combined
        elif model_name == "cnn_combined":
            tok_path = paths["word_tokenizer"]
            if os.path.exists(tok_path) and not force_retrain:
                word_tok = load_tokenizer_json(tok_path)
            else:
                word_tok = build_word_tokenizer(texts, CFG["word"]["max_words"])
                save_tokenizer_json(word_tok, tok_path)
            seqs = word_tok.texts_to_sequences(texts)
            X_word = pad_sequences(seqs, maxlen=CFG["word"]["max_len"], padding="post", truncating="post")
            char_path = paths["char_tokenizer"]
            if os.path.exists(char_path) and not force_retrain:
                char_map, vocab_size = load_char_tokenizer(char_path)
            else:
                char_map = build_char_tokenizer(texts, CFG["char"]["vocab_size"])
                vocab_size = max(char_map.values()) + 1
                save_char_tokenizer(char_map, vocab_size, char_path)
            X_char = texts_to_char_sequences(texts, char_map, CFG["char"]["max_chars"])
            if os.path.exists(paths["model_best"]) and not force_retrain:
                model = load_model(paths["model_best"])
            else:
                word_conf = {"max_len": CFG["word"]["max_len"], "max_words": CFG["word"]["max_words"], "embedding_dim": CFG["word"]["embedding_dim"]}
                char_conf = {"max_chars": CFG["char"]["max_chars"], "vocab_size": vocab_size, "embedding_dim": CFG["char"]["embedding_dim"], "filters": CFG["char"].get("filters",[3,4,5]), "dropout": CFG["char"].get("dropout", CFG["cnn"]["dropout"])}
                model = build_combined_cnn(word_conf, char_conf, n_classes)
            if initial_epoch < epochs:
                model.fit([X_word, X_char], y_cat, batch_size=CFG["training"]["batch_size"], epochs=epochs, initial_epoch=initial_epoch, validation_split=CFG["training"]["validation_split"], callbacks=[cb_epoch, cb_best, cb_early, cb_state], verbose=2)

        # 4) cnn_fasttext_keras
        elif model_name == "cnn_fasttext_keras":
            # Keras-style fasttext: we use a trainable Embedding on word indices (no gensim)
            tok_path = paths["word_tokenizer"]
            if os.path.exists(tok_path) and not force_retrain:
                word_tok = load_tokenizer_json(tok_path)
            else:
                word_tok = build_word_tokenizer(texts, CFG["word"]["max_words"])
                save_tokenizer_json(word_tok, tok_path)
            seqs = word_tok.texts_to_sequences(texts)
            vocab_size = min(CFG["word"]["max_words"], (len(word_tok.word_index) + 1))
            X_word = pad_sequences(seqs, maxlen=CFG["word"]["max_len"], padding="post", truncating="post")
            # char tokenizer for combined branch
            char_path = paths["char_tokenizer"]
            if os.path.exists(char_path) and not force_retrain:
                char_map, _ = load_char_tokenizer(char_path)
            else:
                char_map = build_char_tokenizer(texts, CFG["char"]["vocab_size"])
                save_char_tokenizer(char_map, max(char_map.values())+1, char_path)
            X_char = texts_to_char_sequences(texts, char_map, CFG["char"]["max_chars"])
            # Model: word branch uses embedding with embedding_dim = fasttext_keras embedding dim
            if os.path.exists(paths["model_best"]) and not force_retrain:
                model = load_model(paths["model_best"])
            else:
                # Build CNN that uses trainable embedding for words (fasttext-like)
                w_in = Input(shape=(CFG["word"]["max_len"],), name="word_input")
                w_emb = Embedding(input_dim=vocab_size, output_dim=CFG["fasttext_keras"]["embedding_dim"], input_length=CFG["word"]["max_len"])(w_in)
                convs = []
                for f in CFG["cnn"]["filters"]:
                    c = Conv1D(filters=CFG["cnn"]["num_filters"], kernel_size=f, activation="relu")(w_emb)
                    c = GlobalMaxPooling1D()(c)
                    convs.append(c)
                w_feat = concatenate(convs) if len(convs) > 1 else convs[0]
                w_feat = Dropout(CFG["cnn"]["dropout"])(w_feat)

                # char branch
                c_in = Input(shape=(CFG["char"]["max_chars"],), name="char_input")
                c_emb = Embedding(input_dim=max(char_map.values())+1, output_dim=CFG["char"]["embedding_dim"], input_length=CFG["char"]["max_chars"])(c_in)
                c_convs = []
                for f in CFG["char"].get("filters",[3,4,5]):
                    c = Conv1D(filters=CFG["cnn"]["num_filters"], kernel_size=f, activation="relu")(c_emb)
                    c = GlobalMaxPooling1D()(c)
                    c_convs.append(c)
                c_feat = concatenate(c_convs) if len(c_convs) > 1 else c_convs[0]
                c_feat = Dropout(CFG["char"].get("dropout", 0.5))(c_feat)

                merged = concatenate([w_feat, c_feat])
                merged = Dropout(0.5)(merged)
                out = Dense(n_classes, activation="softmax")(merged)
                model = Model(inputs=[w_in, c_in], outputs=out)
                model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

            if initial_epoch < epochs:
                model.fit([X_word, X_char], y_cat, batch_size=CFG["training"]["batch_size"], epochs=epochs, initial_epoch=initial_epoch, validation_split=CFG["training"]["validation_split"], callbacks=[cb_epoch, cb_best, cb_early, cb_state], verbose=2)

        # 5) cnn_fasttext_gensim
        elif model_name == "cnn_fasttext_gensim":
            ft_path = model_paths(model_name)["fasttext_gensim"]
            sentences = [tokenize_for_fasttext(t) for t in texts]
            if os.path.exists(ft_path) and not force_retrain:
                ft = load_fasttext_gensim(ft_path)
            else:
                ft = train_fasttext_gensim(sentences, ft_path, CFG["fasttext_gensim"])
            embed_dim = ft.vector_size
            max_len = CFG["word"]["max_len"]
            X_word = np.stack([text_to_word_vectors(ft, tokenize_for_fasttext(t), max_len) for t in texts], axis=0)
            char_path = paths["char_tokenizer"]
            if os.path.exists(char_path) and not force_retrain:
                char_map, vocab_size = load_char_tokenizer(char_path)
            else:
                char_map = build_char_tokenizer(texts, CFG["char"]["vocab_size"])
                vocab_size = max(char_map.values()) + 1
                save_char_tokenizer(char_map, vocab_size, char_path)
            X_char = texts_to_char_sequences(texts, char_map, CFG["char"]["max_chars"])
            if os.path.exists(paths["model_best"]) and not force_retrain:
                model = load_model(paths["model_best"])
            else:
                char_conf = {"max_chars": CFG["char"]["max_chars"], "vocab_size": vocab_size, "embedding_dim": CFG["char"]["embedding_dim"], "filters": CFG["char"].get("filters",[3,4,5]), "dropout": CFG["char"].get("dropout", CFG["cnn"]["dropout"])}
                model = build_fasttext_combined_cnn(embed_dim, CFG["word"]["max_len"], char_conf, n_classes)
            if initial_epoch < epochs:
                model.fit([X_word, X_char], y_cat, batch_size=CFG["training"]["batch_size"], epochs=epochs, initial_epoch=initial_epoch, validation_split=CFG["training"]["validation_split"], callbacks=[cb_epoch, cb_best, cb_early, cb_state], verbose=2)

        # 6) rnn_word
        elif model_name == "rnn_word":
            tok_path = paths["word_tokenizer"]
            if os.path.exists(tok_path) and not force_retrain:
                word_tok = load_tokenizer_json(tok_path)
            else:
                word_tok = build_word_tokenizer(texts, CFG["word"]["max_words"])
                save_tokenizer_json(word_tok, tok_path)
            seqs = word_tok.texts_to_sequences(texts)
            X_word = pad_sequences(seqs, maxlen=CFG["word"]["max_len"], padding="post", truncating="post")
            if os.path.exists(paths["model_best"]) and not force_retrain:
                model = load_model(paths["model_best"])
            else:
                model = build_word_rnn(CFG["word"]["max_words"], CFG["word"]["max_len"], CFG["word"]["embedding_dim"], n_classes, CFG["rnn"]["rnn_units"], CFG["rnn"]["dropout"])
            if initial_epoch < epochs:
                model.fit(X_word, y_cat, batch_size=CFG["training"]["batch_size"], epochs=epochs, initial_epoch=initial_epoch, validation_split=CFG["training"]["validation_split"], callbacks=[cb_epoch, cb_best, cb_early, cb_state], verbose=2)

        # 7) rnn_char
        elif model_name == "rnn_char":
            char_path = paths["char_tokenizer"]
            if os.path.exists(char_path) and not force_retrain:
                char_map, vocab_size = load_char_tokenizer(char_path)
            else:
                char_map = build_char_tokenizer(texts, CFG["char"]["vocab_size"])
                vocab_size = max(char_map.values()) + 1
                save_char_tokenizer(char_map, vocab_size, char_path)
            X_char = texts_to_char_sequences(texts, char_map, CFG["char"]["max_chars"])
            if os.path.exists(paths["model_best"]) and not force_retrain:
                model = load_model(paths["model_best"])
            else:
                model = build_char_rnn(vocab_size, CFG["char"]["max_chars"], CFG["char"]["embedding_dim"], CFG["rnn"]["rnn_units"], CFG["rnn"]["dropout"], n_classes)
            if initial_epoch < epochs:
                model.fit(X_char, y_cat, batch_size=CFG["training"]["batch_size"], epochs=epochs, initial_epoch=initial_epoch, validation_split=CFG["training"]["validation_split"], callbacks=[cb_epoch, cb_best, cb_early, cb_state], verbose=2)

        # 8) rnn_combined
        elif model_name == "rnn_combined":
            tok_path = paths["word_tokenizer"]
            if os.path.exists(tok_path) and not force_retrain:
                word_tok = load_tokenizer_json(tok_path)
            else:
                word_tok = build_word_tokenizer(texts, CFG["word"]["max_words"])
                save_tokenizer_json(word_tok, tok_path)
            seqs = word_tok.texts_to_sequences(texts)
            X_word = pad_sequences(seqs, maxlen=CFG["word"]["max_len"], padding="post", truncating="post")
            char_path = paths["char_tokenizer"]
            if os.path.exists(char_path) and not force_retrain:
                char_map, vocab_size = load_char_tokenizer(char_path)
            else:
                char_map = build_char_tokenizer(texts, CFG["char"]["vocab_size"])
                vocab_size = max(char_map.values()) + 1
                save_char_tokenizer(char_map, vocab_size, char_path)
            X_char = texts_to_char_sequences(texts, char_map, CFG["char"]["max_chars"])
            if os.path.exists(paths["model_best"]) and not force_retrain:
                model = load_model(paths["model_best"])
            else:
                word_conf = {"max_len": CFG["word"]["max_len"], "max_words": CFG["word"]["max_words"], "embedding_dim": CFG["word"]["embedding_dim"]}
                char_conf = {"max_chars": CFG["char"]["max_chars"], "vocab_size": vocab_size, "embedding_dim": CFG["char"]["embedding_dim"], "rnn_units": CFG["rnn"]["rnn_units"], "dropout": CFG["rnn"]["dropout"]}
                model = build_combined_rnn(word_conf, char_conf, n_classes)
            if initial_epoch < epochs:
                model.fit([X_word, X_char], y_cat, batch_size=CFG["training"]["batch_size"], epochs=epochs, initial_epoch=initial_epoch, validation_split=CFG["training"]["validation_split"], callbacks=[cb_epoch, cb_best, cb_early, cb_state], verbose=2)

        # 9) rnn_fasttext_keras
        elif model_name == "rnn_fasttext_keras":
            tok_path = paths["word_tokenizer"]
            if os.path.exists(tok_path) and not force_retrain:
                word_tok = load_tokenizer_json(tok_path)
            else:
                word_tok = build_word_tokenizer(texts, CFG["word"]["max_words"])
                save_tokenizer_json(word_tok, tok_path)
            seqs = word_tok.texts_to_sequences(texts)
            vocab_size = min(CFG["word"]["max_words"], (len(word_tok.word_index) + 1))
            X_word = pad_sequences(seqs, maxlen=CFG["word"]["max_len"], padding="post", truncating="post")
            char_path = paths["char_tokenizer"]
            if os.path.exists(char_path) and not force_retrain:
                char_map, _ = load_char_tokenizer(char_path)
            else:
                char_map = build_char_tokenizer(texts, CFG["char"]["vocab_size"])
                save_char_tokenizer(char_map, max(char_map.values())+1, char_path)
            X_char = texts_to_char_sequences(texts, char_map, CFG["char"]["max_chars"])
            if os.path.exists(paths["model_best"]) and not force_retrain:
                model = load_model(paths["model_best"])
            else:
                # RNN combined with trainable embedding for words
                w_in = Input(shape=(CFG["word"]["max_len"],), name="word_input")
                w_emb = Embedding(input_dim=vocab_size, output_dim=CFG["fasttext_keras"]["embedding_dim"], input_length=CFG["word"]["max_len"])(w_in)
                w_x = Bidirectional(LSTM(CFG["rnn"]["rnn_units"]))(w_emb)
                w_x = Dropout(CFG["rnn"]["dropout"])(w_x)

                c_in = Input(shape=(CFG["char"]["max_chars"],), name="char_input")
                c_emb = Embedding(input_dim=max(char_map.values())+1, output_dim=CFG["char"]["embedding_dim"], input_length=CFG["char"]["max_chars"])(c_in)
                c_x = Bidirectional(LSTM(CFG["rnn"]["rnn_units"]))(c_emb)
                c_x = Dropout(CFG["rnn"]["dropout"])(c_x)

                merged = concatenate([w_x, c_x])
                merged = Dropout(0.5)(merged)
                out = Dense(n_classes, activation="softmax")(merged)
                model = Model(inputs=[w_in, c_in], outputs=out)
                model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
            if initial_epoch < epochs:
                model.fit([X_word, X_char], y_cat, batch_size=CFG["training"]["batch_size"], epochs=epochs, initial_epoch=initial_epoch, validation_split=CFG["training"]["validation_split"], callbacks=[cb_epoch, cb_best, cb_early, cb_state], verbose=2)

        # 10) rnn_fasttext_gensim
        elif model_name == "rnn_fasttext_gensim":
            ft_path = model_paths(model_name)["fasttext_gensim"]
            sentences = [tokenize_for_fasttext(t) for t in texts]
            if os.path.exists(ft_path) and not force_retrain:
                ft = load_fasttext_gensim(ft_path)
            else:
                ft = train_fasttext_gensim(sentences, ft_path, CFG["fasttext_gensim"])
            embed_dim = ft.vector_size
            max_len = CFG["word"]["max_len"]
            X_word = np.stack([text_to_word_vectors(ft, tokenize_for_fasttext(t), max_len) for t in texts], axis=0)
            char_path = paths["char_tokenizer"]
            if os.path.exists(char_path) and not force_retrain:
                char_map, vocab_size = load_char_tokenizer(char_path)
            else:
                char_map = build_char_tokenizer(texts, CFG["char"]["vocab_size"])
                vocab_size = max(char_map.values()) + 1
                save_char_tokenizer(char_map, vocab_size, char_path)
            X_char = texts_to_char_sequences(texts, char_map, CFG["char"]["max_chars"])
            if os.path.exists(paths["model_best"]) and not force_retrain:
                model = load_model(paths["model_best"])
            else:
                model = build_fasttext_combined_rnn(embed_dim, CFG["word"]["max_len"], {"max_chars": CFG["char"]["max_chars"], "vocab_size": vocab_size, "embedding_dim": CFG["char"]["embedding_dim"], "rnn_units": CFG["rnn"]["rnn_units"], "dropout": CFG["rnn"]["dropout"]}, n_classes)
            if initial_epoch < epochs:
                model.fit([X_word, X_char], y_cat, batch_size=CFG["training"]["batch_size"], epochs=epochs, initial_epoch=initial_epoch, validation_split=CFG["training"]["validation_split"], callbacks=[cb_epoch, cb_best, cb_early, cb_state], verbose=2)

        else:
            raise ValueError("Unhandled model: " + model_name)

    except Exception as exc:
        tb = traceback.format_exc()
        append_log(model_name, f"TRAIN ERROR: {str(exc)}\n{tb}")
        print("Error training", model_name, ":", str(exc))
        raise

    append_log(model_name, f"=== FINISHED TRAIN [{model_name}] ===")
    print(f"Training finished for: {model_name}")
    return True

# -----------------------------
# Resources & prediction
# -----------------------------
def load_resources_for_model(model_name):
    if model_name not in MODEL_NAMES:
        raise ValueError("Unknown model: " + model_name)
    paths = model_paths(model_name)
    if not os.path.exists(paths["classes"]):
        raise FileNotFoundError("Classes file missing. Train first.")
    classes = np.load(paths["classes"], allow_pickle=True)
    if not os.path.exists(paths["model_best"]):
        raise FileNotFoundError("Best model missing. Train first.")
    model = load_model(paths["model_best"])
    res = {"classes": classes, "model": model}

    if model_name in ["cnn_word", "rnn_word", "cnn_combined", "rnn_combined", "cnn_fasttext_keras", "rnn_fasttext_keras"]:
        if not os.path.exists(paths["word_tokenizer"]):
            raise FileNotFoundError("Word tokenizer missing.")
        res["word_tokenizer"] = load_tokenizer_json(paths["word_tokenizer"])

    if model_name in ["cnn_char", "rnn_char", "cnn_combined", "rnn_combined", "cnn_fasttext_keras", "cnn_fasttext_gensim", "rnn_fasttext_keras", "rnn_fasttext_gensim"]:
        if not os.path.exists(paths["char_tokenizer"]):
            raise FileNotFoundError("Char tokenizer missing.")
        char_map, _ = load_char_tokenizer(paths["char_tokenizer"])
        res["char_map"] = char_map

    if model_name in ["cnn_fasttext_gensim", "rnn_fasttext_gensim", "cnn_fasttext_gensim"]:
        ft_path = model_paths(model_name)["fasttext_gensim"]
        if not os.path.exists(ft_path):
            raise FileNotFoundError("FastText gensim model missing.")
        res["fasttext"] = load_fasttext_gensim(ft_path)

    return res

def predict_for_model(model_name, text, resources):
    t = simple_clean_text(text)
    model = resources["model"]
    classes = resources["classes"]
    if model_name in ["cnn_word", "rnn_word"]:
        tok = resources["word_tokenizer"]
        seq = tok.texts_to_sequences([t])
        x = pad_sequences(seq, maxlen=CFG["word"]["max_len"], padding="post", truncating="post")
        preds = model.predict(x, verbose=0)
    elif model_name in ["cnn_char", "rnn_char"]:
        cm = resources["char_map"]
        x = texts_to_char_sequences([t], cm, CFG["char"]["max_chars"])
        preds = model.predict(x, verbose=0)
    elif model_name in ["cnn_combined", "rnn_combined"]:
        tok = resources["word_tokenizer"]
        seq = tok.texts_to_sequences([t])
        xw = pad_sequences(seq, maxlen=CFG["word"]["max_len"], padding="post", truncating="post")
        cm = resources["char_map"]
        xc = texts_to_char_sequences([t], cm, CFG["char"]["max_chars"])
        preds = model.predict([xw, xc], verbose=0)
    elif model_name in ["cnn_fasttext_keras", "rnn_fasttext_keras"]:
        tok = resources["word_tokenizer"]
        seq = tok.texts_to_sequences([t])
        xw = pad_sequences(seq, maxlen=CFG["word"]["max_len"], padding="post", truncating="post")
        cm = resources["char_map"]
        xc = texts_to_char_sequences([t], cm, CFG["char"]["max_chars"])
        preds = model.predict([xw, xc], verbose=0)
    elif model_name in ["cnn_fasttext_gensim", "rnn_fasttext_gensim"]:
        ft = resources["fasttext"]
        tokens = tokenize_for_fasttext(t)
        xw = np.expand_dims(text_to_word_vectors(ft, tokens, CFG["word"]["max_len"]), axis=0)
        cm = resources["char_map"]
        xc = texts_to_char_sequences([t], cm, CFG["char"]["max_chars"])
        preds = model.predict([xw, xc], verbose=0)
    else:
        raise ValueError("Unknown model: " + model_name)

    idx = int(np.argmax(preds, axis=1)[0])
    prob = float(np.max(preds))
    label = str(classes[idx])
    accepted = (label == "ኖርማል")
    return {"label": label, "probability": prob, "accepted": accepted}

# -----------------------------
# Gradio UI
# -----------------------------
def build_gradio():
    def on_train(model_name, csv_path, force):
        try:
            train_model(model_name, csv_path=csv_path.strip() if csv_path else "", force_retrain=force)
            return f"Training finished for: {model_name}"
        except Exception as e:
            return "ERROR: " + str(e)

    def on_predict(model_name, name):
        try:
            res = load_resources_for_model(model_name)
            r = predict_for_model(model_name, name, res)
            label = r["label"]
            conf = f"{r['probability']*100:.2f}%"
            st = "ACCEPTED ✅" if r["accepted"] else "REJECTED ❌"
            return label, conf, st
        except Exception as e:
            return "ERROR: " + str(e), "", ""

    with gr.Blocks() as demo:
        gr.Markdown("## Unified Models — select a model, train/resume, or predict")
        with gr.Row():
            model_select = gr.Dropdown(MODEL_NAMES, value=MODEL_NAMES[0], label="Model")
            csv_input = gr.Textbox(label="CSV path (leave empty to use fallback)", value="")
        with gr.Row():
            train_btn = gr.Button("Train / Resume Selected Model")
            force_chk = gr.Checkbox(label="Force rebuild (delete/load fresh)", value=False)
            status = gr.Textbox(label="Status", interactive=False)
        with gr.Row():
            name_input = gr.Textbox(label="Proposed Trade Name")
            predict_btn = gr.Button("Predict")
        with gr.Row():
            out_label = gr.Textbox(label="Predicted Reason")
            out_conf = gr.Textbox(label="Confidence")
            out_status = gr.Textbox(label="Decision")

        train_btn.click(on_train, inputs=[model_select, csv_input, force_chk], outputs=[status])
        predict_btn.click(on_predict, inputs=[model_select, name_input], outputs=[out_label, out_conf, out_status])

    return demo

# -----------------------------
# CLI
# -----------------------------
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--csv', default='', help='Path to CSV dataset (optional)')
    parser.add_argument('--train', type=str, help='Train a specific model (name)')
    parser.add_argument('--force', action='store_true', help='Force rebuild')
    parser.add_argument('--serve', action='store_true', help='Launch Gradio UI')
    args = parser.parse_known_args()[0]

    if args.train:
        print("Training:", args.train)
        train_model(args.train, csv_path=args.csv, force_retrain=args.force)
    elif args.serve:
        demo = build_gradio()
        demo.launch()
    else:
        print("Script ready. Use --train <model_name> or --serve to launch the UI.")

if __name__ == "__main__":
    main()
